# A-Cost of pollution

In [1]:
import pandas as pd

In [2]:
dfcost = pd.read_csv("Cost of pollution EU (2021)_update2.xlsx - Cost of pollution EU (2021).csv")

In [3]:
dfcost.head()

,Facility_INSPIRE_ID,facilityName,city,region,Country,sector_en,activity_en,activity_translated,cost_2021
0,PL.MŚ/000000349.FACILITY,PGE Górnictwo i Energetyka Konwencjonalna S.A....,Rogowiec,województwo łódzkie,Poland,Energy sector,Thermal power stations and other combustion in...,Elektrociepłownie i inne instalacje do spalani...,5.618.554.368
1,https://registry.gdi-de.org/id/de.nw.inspire.p...,RWE Power AG - Kraftwerk Neurath,Grevenbroich,Nordrhein-Westfalen,Germany,Energy sector,Thermal power stations and other combustion in...,Wärmekraftwerke und andere Verbrennungsanlagen,3.109.096.192
2,https://registry.gdi-de.org/id/de.bb.inspire.p...,"LEAG, Kraftwerk Jänschwalde",Teichland,Brandenburg,Germany,Energy sector,Thermal power stations and other combustion in...,Wärmekraftwerke und andere Verbrennungsanlagen,2.674.206.976
3,https://registry.gdi-de.org/id/de.sn.sax4inspi...,Kraftwerk Boxberg,Boxberg/O.L.,Sachsen,Germany,Energy sector,Thermal power stations and other combustion in...,Wärmekraftwerke und andere Verbrennungsanlagen,2.500.956.160
4,https://registry.gdi-de.org/id/de.nw.inspire.p...,RWE Power AG Kraftwerk Niederaußem,Bergheim,Nordrhein-Westfalen,Germany,Energy sector,Thermal power stations and other combustion in...,Wärmekraftwerke und andere Verbrennungsanlagen,2.427.463.680


## 1) Cleaning
Dropping mostly useless columns

In [4]:
dfcost = dfcost.drop(["Facility_INSPIRE_ID", "activity_translated"], axis='columns') 

### Cleaning the cost column
Originally done this in the BE df. Pushing code up so it cleans the global df

In [5]:
dfcost["cost_2021"] = dfcost["cost_2021"].str.replace('.', '') 
dfcost["cost_2021"] = dfcost["cost_2021"].fillna(0)
dfcost["cost_2021"] = dfcost["cost_2021"].astype(int)

## 2) Only Belgium 
Keeping original df for future comparison

In [6]:
dfcostbe = dfcost.loc[dfcost["Country"] == "Belgium"]

In [7]:
dfcostbe.head()

,facilityName,city,region,Country,sector_en,activity_en,cost_2021
21,ARCELORMITTAL BELGIUM - GENT,Gent,Flandre orientale,Belgium,Production and processing of metals,Installations for the production of pig iron o...,983884928
30,TotalEnergies Refinery Antwerp,Antwerpen,Anvers,Belgium,Energy sector,Mineral oil and gas refineries,794067904
34,AHLSTROM MUNKSJÖ MALMEDY,Malmedy,Liège,Belgium,Paper and wood production and processing,Industrial plants for the production of paper ...,694007232
63,Electrabel - Centrale Knippegroen,Gent,Flandre orientale,Belgium,Energy sector,Thermal power stations and other combustion in...,429914944
74,BASF ANTWERPEN,Antwerpen,Anvers,Belgium,Chemical industry,Chemical installations for the production on a...,392816128


In [8]:
dfcostbe.to_csv("pollution_cost_be.csv", index=False)

## Per territory

In [9]:
dfcostbe.region.value_counts().sort_values(ascending=False)  #Regions are wrong, would require manual cleaning. 

region
Anvers                 55
Flandre-Occidentale    38
Région wallonne        29
Région flamande        26
Flandre orientale      15
Limbourg               14
Liège                  11
Hainaut                 9
Luxembourg              6
Brabant flamand         6
Bruxelles-Capitale      3
Province de Namur       1
Brabant wallon          1
Name: count, dtype: int64

In [10]:
dfcostbe.groupby("city")["cost_2021"].sum().sort_values(ascending=False)  #Cities make sense

city
Antwerpen                2226697179
Gent                     1859324153
Malmedy                   694007232
Obourg                    269452000
Gaurain-Ramecroix         235492576
                            ...    
Langemark-Poelkapelle        694604
Ledegem                      687726
Tielt                        634237
Vaux-Sous-Chèvremont          57171
Ieper                          5277
Name: cost_2021, Length: 125, dtype: int64

In [11]:
dfcitycostbe = dfcostbe.groupby("city")["cost_2021"].sum().sort_values(ascending=False)

In [12]:
dfcitycostbe.to_csv("pollution_cost_be_per_city.csv")

## Analysis

In [13]:
dfcostbe.cost_2021.sum()

7996066490

In [14]:
dfcostbe.describe()

,cost_2021
count,2.140000e+02
mean,3.736480e+07
std,1.121150e+08
min,5.277000e+03
25%,8.530872e+05
50%,2.354905e+06
75%,2.757097e+07
max,9.838849e+08


Notes:
214 facilities cost > 7,996 million
The data shows a wide range of costs, from a minimum of 5,277 to a maximum of nearly 1 billion.
The mean and median are quite different (mean is much higher than the median), suggesting that the data is skewed to the right (a few very high costs are pulling the mean up).
The large standard deviation relative to the mean further indicates high variability in the costs.

In [15]:
dfcostbe[dfcostbe["cost_2021"] > 27570969]["cost_2021"].sum()


7212190236

In [16]:
dfcostbe[dfcostbe["cost_2021"] > 27570969].count()

facilityName    54
city            53
region          54
Country         54
sector_en       54
activity_en     54
cost_2021       54
dtype: int64

Conclusion: 54 facilities are responsible for 90% of pollution costs 

In [17]:
# Doubling value
dfcostbe[dfcostbe["cost_2021"] > 55141938]["cost_2021"].sum()

6492573940

In [18]:
dfcostbe[dfcostbe["cost_2021"] > 55141938].count()

facilityName    33
city            33
region          33
Country         33
sector_en       33
activity_en     33
cost_2021       33
dtype: int64

Conclusion: 33 facilities are responsible for 81% of pollution costs 

In [19]:
dfcostbe.sort_values(by="cost_2021", ascending=False).head(30)

,facilityName,city,region,Country,sector_en,activity_en,cost_2021
21,ARCELORMITTAL BELGIUM - GENT,Gent,Flandre orientale,Belgium,Production and processing of metals,Installations for the production of pig iron o...,983884928
30,TotalEnergies Refinery Antwerp,Antwerpen,Anvers,Belgium,Energy sector,Mineral oil and gas refineries,794067904
34,AHLSTROM MUNKSJÖ MALMEDY,Malmedy,Liège,Belgium,Paper and wood production and processing,Industrial plants for the production of paper ...,694007232
63,Electrabel - Centrale Knippegroen,Gent,Flandre orientale,Belgium,Energy sector,Thermal power stations and other combustion in...,429914944
74,BASF ANTWERPEN,Antwerpen,Anvers,Belgium,Chemical industry,Chemical installations for the production on a...,392816128
104,EXXONMOBIL PETROLEUM & CHEMICAL - ESSO RAFFINA...,Antwerpen,Anvers,Belgium,Energy sector,Mineral oil and gas refineries,315024288
125,HOLCIM Belgique - Usine d'OBOURG,Obourg,Région wallonne,Belgium,Mineral industry,Installations for the production of cement cli...,269452000
133,ELECTRABEL CENTRALE RODENHUIZE,Gent,Flandre orientale,Belgium,Energy sector,Thermal power stations and other combustion in...,255204144
145,CCB - Site de Gaurain-Ramecroix,Gaurain-Ramecroix,Région wallonne,Belgium,Mineral industry,Installations for the production of cement cli...,235492576
194,CBR sa - Site d'Antoing,Antoing,Région wallonne,Belgium,Mineral industry,Installations for the production of cement cli...,195195872


In [20]:
dfcostbe.groupby("facilityName")["cost_2021"].sum().sort_values(ascending=False).head(30) #Head can be changed to include more or less

facilityName
ARCELORMITTAL BELGIUM - GENT                           983884928
TotalEnergies Refinery Antwerp                         794067904
AHLSTROM  MUNKSJÖ MALMEDY                              694007232
Electrabel - Centrale Knippegroen                      429914944
BASF ANTWERPEN                                         392816128
EXXONMOBIL PETROLEUM & CHEMICAL - ESSO RAFFINADERIJ    315024288
HOLCIM Belgique - Usine d'OBOURG                       269452000
ELECTRABEL CENTRALE RODENHUIZE                         255204144
CCB - Site de Gaurain-Ramecroix                        235492576
CBR sa - Site d'Antoing                                195195872
CBR sa - Site de Lixhe                                 180051568
BURGO ARDENNES                                         178010832
UMICORE - HOBOKEN                                      127319424
AIR LIQUIDE LARGE INDUSTRY                             117401352
TotalEnergies Olefins Antwerp                          107426384
ELECTRABEL -

In [21]:
dfcostbe.groupby("facilityName")["cost_2021"].sum().sort_values(ascending=False).head(30).sum()

6369953881

Conclusion: 30 facilities are responsible for 80% (79.66%) of pollution costs

### Per territory and sector
Preparing data for visualisation

In [22]:
top30 = dfcostbe.sort_values(by="cost_2021", ascending=False).head(30)

In [23]:
#1.Metals
top30[top30["sector_en"] == "Production and processing of metals"]

,facilityName,city,region,Country,sector_en,activity_en,cost_2021
21,ARCELORMITTAL BELGIUM - GENT,Gent,Flandre orientale,Belgium,Production and processing of metals,Installations for the production of pig iron o...,983884928
327,UMICORE - HOBOKEN,Antwerpen,Anvers,Belgium,Production and processing of metals,Installation for the production of non-ferrous...,127319424


In [24]:
#2. Energy
top30[top30["sector_en"] == "Energy sector"]

,facilityName,city,region,Country,sector_en,activity_en,cost_2021
30,TotalEnergies Refinery Antwerp,Antwerpen,Anvers,Belgium,Energy sector,Mineral oil and gas refineries,794067904
63,Electrabel - Centrale Knippegroen,Gent,Flandre orientale,Belgium,Energy sector,Thermal power stations and other combustion in...,429914944
104,EXXONMOBIL PETROLEUM & CHEMICAL - ESSO RAFFINA...,Antwerpen,Anvers,Belgium,Energy sector,Mineral oil and gas refineries,315024288
133,ELECTRABEL CENTRALE RODENHUIZE,Gent,Flandre orientale,Belgium,Energy sector,Thermal power stations and other combustion in...,255204144
426,ELECTRABEL - CENTRALE D'AMERCOEUR,Roux,Région wallonne,Belgium,Energy sector,Thermal power stations and other combustion in...,101475648
541,INDUSTRIE DU BOIS VIELSALM & CIE - IBV,Vielsalm,Luxembourg,Belgium,Energy sector,Thermal power stations and other combustion in...,81751544
557,ZANDVLIET POWER - TERREIN BASF,Antwerpen,Anvers,Belgium,Energy sector,Thermal power stations and other combustion in...,79335072
652,LUMINUS,Gent,Flandre orientale,Belgium,Energy sector,Thermal power stations and other combustion in...,67063012
662,T POWER,Tessenderlo,Limbourg,Belgium,Energy sector,Thermal power stations and other combustion in...,65962304
715,ELECTRABEL - CENTRALE BAUDOUR/St GHISLAIN,Baudour,Luxembourg,Belgium,Energy sector,Thermal power stations and other combustion in...,59909400


(I left this part of the analysis on standby)

# B- Belgium's pollution over the years

In [25]:
dftime = pd.read_csv("Belgium.xlsx - Belgium.csv")

In [26]:
dftime.head()

,Facility_INSPIRE_ID,facilityName,parentCompanyName,city,region,sector,activity_en,activity_translated,countryCode,pollutant,...,2014 (kg),2015 (kg),2016 (kg),2017 (kg),2018 (kg),2019 (kg),2020 (kg),2021 (kg),2022 (kg),All years (kg)
0,BE.BRU/100010004.FACILITY,BRUXELLES ENERGIE,Bruxelles Propreté - Agence Régionale pour la ...,Bruxelles,Bruxelles-Capitale,Gestion des déchets et des eaux usées,Installations for the incineration of non-haza...,Installations destinées à l'incinération des d...,BE,Nitrogen oxides (NOX),...,158000.0,163000.0,127000.0,134000.0,106000.0,156000.0,150000.0,124000.0,98300.0,2827300
1,BE.BRU/100010004.FACILITY,BRUXELLES ENERGIE,Bruxelles Propreté - Agence Régionale pour la ...,Bruxelles,Bruxelles-Capitale,Gestion des déchets et des eaux usées,Installations for the incineration of non-haza...,Installations destinées à l'incinération des d...,BE,Carbon dioxide (CO2),...,484000000.0,473000000.0,492000000.0,427000000.0,485000000.0,625000000.0,442000000.0,471000000.0,504000000.0,7442000000
2,BE.BRU/100010006.FACILITY,AUDI BRUSSELS,Audi AG,Forest,Bruxelles-Capitale,Production et transformation des métaux,Installations for surface treatment of metals ...,Installations de traitement de surface des mét...,BE,Non-methane volatile organic compounds (NMVOC),...,322000.0,320000.0,277000.0,244000.0,122000.0,118000.0,NaN,NaN,104000.0,3495000
3,BE.BRU/100010015.FACILITY,STEP SUD (SBGE),SBGE,Forest,Bruxelles-Capitale,Gestion des déchets et des eaux usées,Urban waste-water treatment plants,Installations de traitement des eaux urbaines ...,BE,Nitrous oxide (N2O),...,NaN,NaN,40100.0,22100.0,NaN,NaN,NaN,NaN,NaN,62200
4,BE.BRU/100010015.FACILITY,STEP SUD (SBGE),SBGE,Forest,Bruxelles-Capitale,Gestion des déchets et des eaux usées,Urban waste-water treatment plants,Installations de traitement des eaux urbaines ...,BE,Sulphur oxides (SOX),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,516000.0,NaN,516000


In [27]:
dftime = dftime.drop(["Facility_INSPIRE_ID", "activity_translated", "countryCode", "pollutant_translated"], axis='columns')

In [28]:
dftime.dtypes

facilityName          object
parentCompanyName     object
city                  object
region                object
sector                object
activity_en           object
pollutant             object
2007 (kg)            float64
2008 (kg)            float64
2009 (kg)            float64
2010 (kg)            float64
2011 (kg)            float64
2012 (kg)            float64
2013 (kg)            float64
2014 (kg)            float64
2015 (kg)            float64
2016 (kg)            float64
2017 (kg)            float64
2018 (kg)            float64
2019 (kg)            float64
2020 (kg)            float64
2021 (kg)            float64
2022 (kg)            float64
All years (kg)         int64
dtype: object

In [29]:
# Selecting only the top30 installation that polluted the most in terms of economical cost, 
# following the previous analysis

top_facs = ["ARCELORMITTAL BELGIUM - GENT", 
"TotalEnergies Refinery Antwerp", 
"AHLSTROM  MUNKSJÖ MALMEDY", 
"Electrabel - Centrale Knippegroen", 
"BASF ANTWERPEN", 
"EXXONMOBIL PETROLEUM & CHEMICAL - ESSO RAFFINADERIJ", 
"HOLCIM Belgique - Usine d'OBOURG", 
"ELECTRABEL CENTRALE RODENHUIZE", 
"CCB - Site de Gaurain-Ramecroix", 
"CBR sa - Site d'Antoing",
"CBR sa - Site de Lixhe",
"BURGO ARDENNES", 
"UMICORE - HOBOKEN",            
"AIR LIQUIDE LARGE INDUSTRY",                      
"TotalEnergies Olefins Antwerp",                       
"ELECTRABEL - CENTRALE D'AMERCOEUR",                   
"INDUSTRIE DU BOIS VIELSALM & CIE - IBV",              
"ARGEX",                                                
"EVONIK ANTWERPEN",                                
"ZANDVLIET POWER - TERREIN BASF",                        
"STORA ENSO LANGERBRUGGE",                              
"EUROCHEM ANTWERPEN",                                  
"WIENERBERGER",                                      
"BIOWANZE",                                      
"LUMINUS",                                             
"T POWER",                                            
"CARRIERES ET FOURS A CHAUX DUMONT WAUTIER",              
"LANXESS",                                               
"ELECTRABEL - CENTRALE BAUDOUR/St GHISLAIN",             
"INOVYN Manufacturing Belgium"]

In [30]:
len(top_facs)

30

In [31]:
dftimetop = dftime.loc[dftime["facilityName"].isin(top_facs)]

In [32]:
len(dftimetop)

137

### By sector

In [33]:
dftimetop["sector"].value_counts()

sector
Industrie minérale                                    44
Secteur énergétique                                   43
Industrie chimique                                    30
Production et transformation des métaux               11
Fabrication et transformation du papier et du bois     9
Name: count, dtype: int64

Creating a macro df with sectors' combined pollution in kg over the years

In [34]:
sector11 = dftimetop.groupby("sector")["2011 (kg)"].sum()

In [35]:
sector12 = dftimetop.groupby("sector")["2012 (kg)"].sum()

In [36]:
sector13 = dftimetop.groupby("sector")["2013 (kg)"].sum()

In [37]:
sector14 = dftimetop.groupby("sector")["2014 (kg)"].sum()

In [38]:
sector15 = dftimetop.groupby("sector")["2015 (kg)"].sum()

In [39]:
sector16 = dftimetop.groupby("sector")["2016 (kg)"].sum()

In [40]:
sector17 = dftimetop.groupby("sector")["2017 (kg)"].sum()

In [41]:
sector18 = dftimetop.groupby("sector")["2018 (kg)"].sum()

In [42]:
sector19 = dftimetop.groupby("sector")["2019 (kg)"].sum()

In [43]:
sector20 = dftimetop.groupby("sector")["2020 (kg)"].sum()

In [44]:
sector21 = dftimetop.groupby("sector")["2021 (kg)"].sum()

Merging sector/year dfs

In [45]:
sector_1121 = pd.merge(sector11, sector12, on="sector")

In [46]:
sector_1121 = pd.merge(sector_1121, sector13, on="sector")

In [47]:
sector_1121 = pd.merge(sector_1121, sector14, on="sector")

In [48]:
sector_1121 = pd.merge(sector_1121, sector15, on="sector")

In [49]:
sector_1121 = pd.merge(sector_1121, sector16, on="sector")

In [50]:
sector_1121 = pd.merge(sector_1121, sector17, on="sector")

In [51]:
sector_1121 = pd.merge(sector_1121, sector18, on="sector")

In [52]:
sector_1121 = pd.merge(sector_1121, sector19, on="sector")

In [53]:
sector_1121 = pd.merge(sector_1121, sector20, on="sector")

In [54]:
sector_1121 = pd.merge(sector_1121, sector21, on="sector")

In [55]:
sector_1121.to_csv("sector_1121.csv")

In [56]:
dftimetop.head()

,facilityName,parentCompanyName,city,region,sector,activity_en,pollutant,2007 (kg),2008 (kg),2009 (kg),...,2014 (kg),2015 (kg),2016 (kg),2017 (kg),2018 (kg),2019 (kg),2020 (kg),2021 (kg),2022 (kg),All years (kg)
23,WIENERBERGER,WIENERBERGER,Tessenderlo,Limbourg,Industrie minérale,Installations for the manufacture of ceramic p...,Sulphur oxides (SOX),710000.0,919000.0,559000.0,...,255000.0,156000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3192000
31,WIENERBERGER,WIENERBERGER,Beerse,Anvers,Industrie minérale,Installations for the manufacture of ceramic p...,Sulphur oxides (SOX),510000.0,321000.0,330000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1549000
130,WIENERBERGER,WIENERBERGER,Pittem,Flandre-Occidentale,Industrie minérale,Installations for the manufacture of ceramic p...,Sulphur oxides (SOX),NaN,374000.0,322000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1195000
257,CBR sa - Site de Lixhe,Heidelbergcement,Lixhe,Région wallonne,Industrie minérale,Installations for the production of cement cli...,Nitrogen oxides (NOX),2750000.0,2670000.0,2030000.0,...,1800000.0,1670000.0,1410000.0,986000.0,960000.0,966000.0,1380000.0,1180000.0,1080000.0,28002000
258,CBR sa - Site de Lixhe,Heidelbergcement,Lixhe,Région wallonne,Industrie minérale,Installations for the production of cement cli...,Ammonia (NH3),28000.0,26100.0,35400.0,...,49700.0,33100.0,18300.0,28700.0,21000.0,21100.0,18000.0,13900.0,12700.0,435300


## Calculations and analysis

Creating table with total pollutants over the years for comparison per installation

In [57]:
df2021 = dftimetop.groupby("facilityName")["2021 (kg)"].sum()

In [58]:
df2016 = dftimetop.groupby("facilityName")["2016 (kg)"].sum()

In [59]:
df2011 = dftimetop.groupby("facilityName")["2011 (kg)"].sum()

In [60]:
dfyears = pd.merge(df2011, df2016, on = 'facilityName')

In [61]:
dfyears = pd.merge(dfyears, df2021, on = 'facilityName')

In [62]:
dfyears.head()

,2011 (kg),2016 (kg),2021 (kg)
facilityName,,,
AIR LIQUIDE LARGE INDUSTRY,7.421610e+08,5.911510e+08,1.028194e+09
ARCELORMITTAL BELGIUM - GENT,3.713661e+09,4.173847e+09,3.651097e+09
ARGEX,1.262000e+06,1.010000e+06,1.224100e+06
BASF ANTWERPEN,2.884586e+09,3.112422e+09,2.962239e+09
BIOWANZE,4.572460e+08,5.142210e+08,5.812269e+08


In [63]:
dfyears.shape

(28, 3)

In [64]:
dfyears.to_csv("polluters_2011_21.csv")

Adding parent company

In [65]:
df_parent = dftime[["facilityName", "parentCompanyName"]]

In [66]:
dfyears = pd.merge(left = dfyears, 
               right = df_parent, 
               how = 'left', 
               on = 'facilityName')

In [67]:
dfyears.head()

,facilityName,2011 (kg),2016 (kg),2021 (kg),parentCompanyName
0,AIR LIQUIDE LARGE INDUSTRY,7.421610e+08,5.911510e+08,1.028194e+09,AIR LIQUIDE LARGE INDUSTRY
1,AIR LIQUIDE LARGE INDUSTRY,7.421610e+08,5.911510e+08,1.028194e+09,AIR LIQUIDE LARGE INDUSTRY
2,AIR LIQUIDE LARGE INDUSTRY,7.421610e+08,5.911510e+08,1.028194e+09,AIR LIQUIDE LARGE INDUSTRY
3,ARCELORMITTAL BELGIUM - GENT,3.713661e+09,4.173847e+09,3.651097e+09,ARCELORMITTAL BELGIUM
4,ARCELORMITTAL BELGIUM - GENT,3.713661e+09,4.173847e+09,3.651097e+09,ARCELORMITTAL BELGIUM


In [68]:
dfyears = dfyears.drop_duplicates() #All were back, so dropping

In [69]:
dfyears.shape

(28, 5)

In [70]:
dfcompany = dfyears.groupby("parentCompanyName").sum()

In [71]:
dfcompany.to_csv("companies_2011_21.csv")

## Including longlat 

In [72]:
ci = pd.read_csv("worldcities.csv")

In [73]:
ci = ci.loc[ci["iso2"] == "BE"]

In [74]:
ci.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
619,Brussels,Brussels,50.8467,4.3525,Belgium,BE,BEL,Brussels-Capital Region,primary,1235192.0,1056469830
1381,Antwerp,Antwerp,51.2178,4.4003,Belgium,BE,BEL,Flanders,minor,536079.0,1056168623
2552,Gent,Gent,51.0536,3.7253,Belgium,BE,BEL,Flanders,minor,265086.0,1056062897
3273,Charleroi,Charleroi,50.4000,4.4333,Belgium,BE,BEL,Wallonia,minor,201816.0,1056263311
3383,Liège,Liege,50.6397,5.5706,Belgium,BE,BEL,Wallonia,minor,195278.0,1056513284


In [75]:
ci = ci.drop(['iso2', 'iso3', 'id'], axis=1)

In [76]:
cols = ["city", "lat", "lng", "population"]
ci_small = ci[cols]    #Keeping original cities cols

In [77]:
ci_small.head()

,city,lat,lng,population
619,Brussels,50.8467,4.3525,1235192.0
1381,Antwerp,51.2178,4.4003,536079.0
2552,Gent,51.0536,3.7253,265086.0
3273,Charleroi,50.4000,4.4333,201816.0
3383,Liège,50.6397,5.5706,195278.0


## Including missing cities
I did this for the top 10, didn't continue for the top 30

In [78]:
new_rows = pd.DataFrame({
    'city': ['Antwerpen', 'Gaurain-Ramecroix', 'Obourg', 'Antoing'],
    'lat': [51.2178, 50.589, 50.476111, 50.5677],
    'lng': [4.4003, 3.488, 4.006111, 3.4492],
    'population': [536079.0, 3619.0, 4488.0, 7760.0]
})

# Adding the new rows
ci_small = pd.concat([ci_small, new_rows], ignore_index=True)


In [79]:
top = pd.merge(left = dftimetop, 
               right = ci_small, 
               how = 'left', 
               on = 'city')

In [80]:
top.sample()

,facilityName,parentCompanyName,city,region,sector,activity_en,pollutant,2007 (kg),2008 (kg),2009 (kg),...,2017 (kg),2018 (kg),2019 (kg),2020 (kg),2021 (kg),2022 (kg),All years (kg),lat,lng,population
118,Electrabel - Centrale Knippegroen,ELECTRABEL,Gent,Flandre orientale,Secteur énergétique,Thermal power stations and other combustion in...,Sulphur oxides (SOX),NaN,NaN,NaN,...,529000.0,341000.0,389000.0,463000.0,453000.0,612000.0,5875000,51.0536,3.7253,265086.0


In [81]:
top[top["lat"].isna()] #No latlng NaNs? OK!

,facilityName,parentCompanyName,city,region,sector,activity_en,pollutant,2007 (kg),2008 (kg),2009 (kg),...,2017 (kg),2018 (kg),2019 (kg),2020 (kg),2021 (kg),2022 (kg),All years (kg),lat,lng,population
2,WIENERBERGER,WIENERBERGER,Pittem,Flandre-Occidentale,Industrie minérale,Installations for the manufacture of ceramic p...,Sulphur oxides (SOX),NaN,3.740000e+05,3.220000e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,1195000,NaN,NaN,NaN
3,CBR sa - Site de Lixhe,Heidelbergcement,Lixhe,Région wallonne,Industrie minérale,Installations for the production of cement cli...,Nitrogen oxides (NOX),2.750000e+06,2.670000e+06,2.030000e+06,...,9.860000e+05,9.600000e+05,9.660000e+05,1.380000e+06,1.180000e+06,1.080000e+06,28002000,NaN,NaN,NaN
4,CBR sa - Site de Lixhe,Heidelbergcement,Lixhe,Région wallonne,Industrie minérale,Installations for the production of cement cli...,Ammonia (NH3),2.800000e+04,2.610000e+04,3.540000e+04,...,2.870000e+04,2.100000e+04,2.110000e+04,1.800000e+04,1.390000e+04,1.270000e+04,435300,NaN,NaN,NaN
5,CBR sa - Site de Lixhe,Heidelbergcement,Lixhe,Région wallonne,Industrie minérale,Installations for the production of cement cli...,Nitrous oxide (N2O),2.670000e+04,2.350000e+04,NaN,...,1.020000e+04,1.480000e+04,1.520000e+04,1.610000e+04,1.560000e+04,4.870000e+04,291200,NaN,NaN,NaN
6,CBR sa - Site de Lixhe,Heidelbergcement,Lixhe,Région wallonne,Industrie minérale,Installations for the production of cement cli...,Non-methane volatile organic compounds (NMVOC),1.160000e+05,1.230000e+05,1.590000e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,604000,NaN,NaN,NaN
7,CBR sa - Site de Lixhe,Heidelbergcement,Lixhe,Région wallonne,Industrie minérale,Installations for the production of cement cli...,Carbon dioxide (CO2),1.220000e+09,1.210000e+09,1.140000e+09,...,1.140000e+09,1.160000e+09,1.190000e+09,1.260000e+09,1.210000e+09,1.180000e+09,18450000000,NaN,NaN,NaN
29,CARRIERES ET FOURS A CHAUX DUMONT WAUTIER,CARRIERES ET FOURS A CHAUX DUMONT-WAUTIER,Saint-Georges-Sur-Meuse,Région wallonne,Industrie minérale,Installations for the production of cement cli...,Nitrous oxide (N2O),NaN,1.180000e+04,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,22300,NaN,NaN,NaN
30,CARRIERES ET FOURS A CHAUX DUMONT WAUTIER,CARRIERES ET FOURS A CHAUX DUMONT-WAUTIER,Saint-Georges-Sur-Meuse,Région wallonne,Industrie minérale,Installations for the production of cement cli...,Nitrogen oxides (NOX),2.250000e+06,1.060000e+06,9.760000e+05,...,1.240000e+06,1.380000e+06,1.190000e+06,3.660000e+05,6.980000e+05,5.020000e+05,18747000,NaN,NaN,NaN
31,CARRIERES ET FOURS A CHAUX DUMONT WAUTIER,CARRIERES ET FOURS A CHAUX DUMONT-WAUTIER,Saint-Georges-Sur-Meuse,Région wallonne,Industrie minérale,Installations for the production of cement cli...,Sulphur oxides (SOX),3.050000e+06,2.170000e+06,5.930000e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,6755000,NaN,NaN,NaN
32,CARRIERES ET FOURS A CHAUX DUMONT WAUTIER,CARRIERES ET FOURS A CHAUX DUMONT-WAUTIER,Saint-Georges-Sur-Meuse,Région wallonne,Industrie minérale,Installations for the production of cement cli...,Carbon dioxide (CO2),1.310000e+09,1.240000e+09,9.000000e+08,...,1.010000e+09,9.850000e+08,8.170000e+08,5.820000e+08,3.960000e+08,5.510000e+08,15781000000,NaN,NaN,NaN


In [82]:
ci_small.to_csv("Belgian_cities.csv", index=False)

In [83]:
dfcostbe.to_csv("pollution_cost_be_per_fac.csv", index=False)